# 0. Library

In [10]:
import os, time
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from tqdm import tqdm
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer


# 1. Config

In [11]:
class Config:
    bos_token = '<s>'
    eos_token = '</s>'
    usr_token = '<usr>'
    pad_token = '<pad>'
    sys_token = '<sys>'
    unk_token = '<unk>'
    mask_token = '<mask>'
    max_length = 256
    max_turns = 8
    epochs = 2
    batch_size = 4
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = 1e-4
    model_name = "skt/kogpt2-base-v2"

# 2. Data

In [12]:
raw_data = pd.read_csv('data/kakao_len256.csv')

In [13]:
N = 1000# len(raw_data)
data = raw_data.sample(N)
data

,conversation
43111,<usr> 헬스장 갔다 사우나 하고 나오니 기분 좋네요<sys> 전 오늘 스카이다이...
48021,<usr> 군대에서 한 달에 한번 행군을 해<sys> 행군? 그게 뭐야? 걷는거?<...
56117,<usr> 넌 흥부와 놀부 중에 누가 좋니?<sys> 당연히 놀부<usr> 특이하네...
20758,<usr> 너 배우 마동석 좋아해?<sys> 응 마동석 너무 좋아하지<usr> 나 ...
50760,<usr> 반려동물 시장이 갈수록 커져<sys> 응. 물건들도 다양해졌어<usr> ...
...,...
48953,<usr> 변요한이 영화공약했었잖아<sys> 관객 얼마 이상되면 춤 추겠다던거?<u...
12048,<usr> 나 요즘 시골 내려가서 살고싶어<sys> 갑자기 왠 시골이야?<usr> ...
51000,<usr> 할인할 때 왕창 사놓는 물건 있어?<sys> 고기 세일하면 사놓는 편이야...
40585,<usr> 얘들아 나 고민 있어.<sys> 무슨 일이야?<usr> 뭔데? 말해봐 봐...


In [14]:
def valid_size(train_X):
    length = len(train_X)
    return round(0.14 * (1 + length / 10 ** 4) ** (10 ** 4 / length) - 0.13, 2)

In [15]:
data_train, data_valid = train_test_split(data, test_size=valid_size(data), shuffle=True)
data_train.reset_index(inplace=True)
data_valid.reset_index(inplace=True)

In [16]:
data_train

,index,conversation
0,47510,<usr> 라라랜드 같은 영화 보고싶다<sys> 라라랜드 완전 명작이잖아 하하<us...
1,36834,<usr> 언니 요즘 뭐 보는 거 있어?<sys> 스우파랑 오징어 게임 봤어 그리고...
2,28561,<usr> 오늘 무신사에서 산 옷 배송 온다<sys> 거기 배송 느리다던데 진짜야?...
3,20282,<usr> 하하 부대 위치는 어디였어?<sys> 나 부대 위치는 송탄이라고 있는데 ...
4,33848,<usr> 오빠는 배그 집에서 안해?<sys> 배그는 하면 눈이 너무 아프더라..<...
...,...,...
765,12820,<usr> 결혼정보회사 가입 어떻게 하지?<sys> 이런 저런 서류들이 필요할 거야...
766,57941,<usr> 언니 예전에 애들 친구들이랑 생일파티 했다 아니예요?<sys> 그래 그 ...
767,36073,<usr> 전기료가 오른다네유..물가 무엇..<sys> 그러게요.. 이래서 전기차 ...
768,36920,<usr> 니 형있다 했제? ㅋㅋ니네 형은 결혼하셨나? ㅋㅋ<sys> 아니요 아직 ...


In [17]:
data_valid

,index,conversation
0,29035,<usr> 놀럭자놀러가자<sys> 좋아좋아<usr> 어디든..<sys> 담달에 추석...
1,10207,<usr> 복도식아파트살면 복도가 자기껀줄 착각하나?<sys> 으..뭔일있는거야 왜...
2,451,<usr> 오징어게임 인기가 오래간다<sys> 그러게 식을 줄을 모르네<usr> 뉴...
3,54976,<usr> 야 나 어제 입원 했다<sys> 갑자기 무슨 일이야?<usr> 어제 일 ...
4,46453,<usr> 글램핑은 대전 내인가요?<sys> 아뇨 논산이래요<usr> 거리는 얼마나...
...,...,...
225,17825,<usr> 신랑이 이사온곳에 미용실을 못뚫어서 ㅋ<sys> 썬크림 바르고 가렴 ㅋㅋ...
226,37154,<usr> 밖에 나왔는데 제법 쌀쌀해<sys> 산에 갔을 땐 더웠는데 내려오니 추워...
227,55356,<usr> 굿플레이스라고 들어봤어?<sys> 아니 그것도 처음 들어봤어<usr> 이...
228,52592,<usr> 베이징에선 벌써 첫눈이 내린 모양이구만<sys> 벌써? 아직 12월도 안...


# 3. Tokenizer

In [18]:
tokenizer = AutoTokenizer.from_pretrained(Config.model_name,
            bos_token=Config.bos_token, eos_token=Config.eos_token,
            unk_token=Config.unk_token, pad_token=Config.pad_token,
            mask_token=Config.mask_token, model_max_length=Config.max_length)

In [19]:
for i in range(10):
    print(tokenizer.convert_ids_to_tokens(i), end=' ')

<s> </s> <usr> <pad> <sys> <unk> <mask> <d> </d> <unused0> 

# 4. Dataloader

In [20]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, Config):
        self.data = data
        self.tokenizer = tokenizer
        self.bos_token = Config.bos_token
        self.eos_token = Config.eos_token
        self.usr_token = Config.usr_token
        self.pad_token = Config.pad_token
        self.sys_token = Config.sys_token

        self.max_length = Config.max_length
        self.max_turns = Config.max_turns
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        sentence = self.data['conversation'][idx]
        # input_id
        input_id = self.tokenizer.encode(self.bos_token + sentence + self.eos_token)
        # token_type_id
        token_type = []
        loop = True
        for token_id in input_id:
            token = self.tokenizer.convert_ids_to_tokens(token_id)
            
            if token == self.usr_token: loop=True
            elif token == self.sys_token: loop=False
                
            if loop:
                token_type.append(self.usr_token)
            else:
                token_type.append(self.sys_token)
        token_type_id = self.tokenizer.convert_tokens_to_ids(token_type)
        # label
        start_idx = len(input_id) - list(reversed(input_id)).index(self.tokenizer.convert_tokens_to_ids(self.sys_token))
        label = [-100] * start_idx + input_id[start_idx: ]
        # padding
        input_id, token_type_id, label = self.make_padding(input_id, token_type_id, label)
        
        return input_id, token_type_id, label

    def make_padding(self, input_id, token_type_id, label):
        left_length = self.max_length - len(input_id)
        input_id += [self.tokenizer.pad_token_id] * left_length
        token_type_id += [self.tokenizer.pad_token_id] * left_length
        label += [-100] * left_length
        
        return input_id, token_type_id, label

In [21]:
train_set = CustomDataset(data_train, tokenizer, Config)

In [22]:
input_id, token_type_id, label = train_set[0]
print("input_id", input_id, sep='\n')
print("token_type_id", token_type_id, sep='\n')
print("label", label, sep='\n')

input_id
[0, 2, 9755, 7372, 9936, 9239, 10584, 10056, 7898, 7182, 4, 9755, 7372, 9936, 10253, 9170, 25138, 8162, 7965, 9078, 8702, 2, 15247, 12102, 9018, 6857, 10584, 17692, 9350, 7662, 7182, 739, 216, 4, 20713, 12858, 30616, 9843, 13454, 9078, 8702, 2, 9198, 23775, 13444, 7991, 376, 13348, 9774, 7623, 8137, 739, 7662, 15433, 10125, 7372, 4, 9769, 9755, 7372, 9936, 10288, 13394, 6824, 12371, 12011, 2, 9050, 9022, 6853, 9034, 6890, 19778, 9867, 9564, 9625, 16693, 4, 15247, 9036, 6890, 8704, 6890, 9054, 12011, 10104, 6919, 739, 216, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [23]:
def collate_fn(batch):
    input_ids = [items[0] for items in batch]
    token_type_ids = [items[1] for items in batch]
    labels = [items[2] for items in batch]
    
    return torch.LongTensor(input_ids), torch.LongTensor(token_type_ids), \
            torch.LongTensor(labels)

In [24]:
train_dataloader = DataLoader(train_set, batch_size=Config.batch_size, num_workers=2,
                              shuffle=True, collate_fn=collate_fn)


In [25]:
valid_set = CustomDataset(data_valid, tokenizer, Config)
valid_dataloader = DataLoader(valid_set, batch_size=Config.batch_size, num_workers=2,
                            shuffle=False, collate_fn=collate_fn)


In [26]:
input_id, token_type_id, label = valid_set[0]
print("input_id", input_id, sep='\n')
print("token_type_id", token_type_id, sep='\n')
print("label", label, sep='\n')

input_id
[0, 2, 10624, 7398, 8159, 7122, 27006, 8159, 4, 12011, 8223, 7965, 2, 13400, 7283, 9705, 4, 9467, 48988, 9220, 7789, 8033, 8791, 7312, 15359, 25594, 9700, 33778, 9330, 19816, 9546, 47804, 7965, 7172, 7532, 11528, 7235, 10586, 7220, 2, 11786, 8101, 16256, 7058, 13400, 7283, 9054, 9677, 6960, 8704, 7220, 28737, 7380, 20130, 9054, 12011, 6949, 7220, 11528, 8135, 9183, 7253, 6853, 6838, 36598, 6953, 12371, 9286, 8265, 8159, 23982, 216, 4, 15247, 9018, 18554, 9286, 8265, 8159, 22683, 7536, 9168, 15182, 7788, 12079, 16179, 6958, 20164, 14520, 9019, 6960, 8704, 7220, 8397, 7789, 7312, 9168, 10307, 7478, 6962, 16098, 9685, 8263, 8163, 9511, 12858, 42351, 10307, 7481, 14520, 9019, 8018, 2, 9320, 9018, 18554, 11528, 8146, 17733, 8519, 18048, 9022, 6853, 15546, 9286, 8265, 8159, 9564, 12964, 7788, 24237, 21716, 7898, 8135, 7220, 9350, 6824, 6872, 8006, 216, 4, 16518, 11528, 7492, 6889, 9306, 6899, 20130, 9955, 8006, 8335, 8694, 9546, 6899, 8135, 9146, 8161, 8710, 9078, 8702, 2, 16256, 22

# 5. Model

In [27]:
model = GPT2LMHeadModel.from_pretrained(Config.model_name).to(Config.device)

# 6. Train

In [28]:
class ChatBot:
    def __init__(self, model, tokenizer, Config):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=Config.learning_rate)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.9)
        self.bos_token = Config.bos_token
        self.eos_token = Config.eos_token
        self.usr_token = Config.usr_token
        self.sys_token = Config.sys_token
        self.max_length = Config.max_length
        self.max_turns = Config.max_turns
        self.device = Config.device
        self.losses = []
        self.val_losses = []
        self.history = []

    def train(self, epochs, train_dataloader, validation_dataloader=None, save=None):
        self.model.train()
        for epoch in range(epochs):
            print(f"\n Epoch {epoch+1}/{epochs}", sep="\n")
            batch_loss = []
            start_time = time.time()

            for i, batch in enumerate(train_dataloader):
                input_ids, token_type_ids, labels = batch        
                input_ids, token_type_ids, labels = input_ids.to(self.device), token_type_ids.to(self.device), \
                                                    labels.to(self.device)
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                batch_loss.append(loss.item())
                
                print(self.status(i+1, len(train_dataloader), time.time()-start_time, np.mean(batch_loss)), end='\r')
            
            self.scheduler.step()
            
            self.losses.append(np.mean(batch_loss))
            
            if validation_dataloader:
                val_loss = self.validation(validation_dataloader)
                print(self.status(i+1, len(train_dataloader), time.time()-start_time, np.mean(batch_loss)) + \
                      " | val_loss : %.6f"%(val_loss), end='\r')
                self.val_losses.append(val_loss)
            
            if save:
                time_zone = datetime.timezone(datetime.timedelta(hours=9))
                now = datetime.datetime.now(time_zone)
                PATH = now.strftime(f'./check_point/%Y-%m-%d-%Hh-%Mm_epoch_{epoch+1}.pth')
                torch.save(self.model.state_dict(), PATH)

    def validation(self, validation_dataloader):
        self.model.eval()
        batch_loss = []
        
        with torch.no_grad():
            for i, batch in enumerate(validation_dataloader):
                input_ids, token_type_ids, labels = batch
                input_ids, token_type_ids, labels = input_ids.to(self.device), token_type_ids.to(self.device),\
                                                    labels.to(self.device)
                
                outputs = self.model(
                    input_ids = input_ids,
                    token_type_ids = token_type_ids,
                    labels = labels
                )
                
                loss = outputs.loss
                batch_loss.append(loss.item())
            
            valid_loss = np.mean(batch_loss)
        
        return valid_loss
    
    @staticmethod
    def status(step, step_len, time, loss):
        return f"step: {step} / {step_len} - {int(time)} s | loss: {loss:.6f} | {step/time:.2f} it/s"

    def save(self, PATH=None):
        if not PATH:
            now = datetime.datetime.now()
            now_date = now.strftime('%m%d_%H%M')
            PATH = 'models/' + str(now_date) + '_model.pt'
        torch.save(self.model.state_dict(), PATH)
        print("model saved.")

    def load(self, PATH):
        self.model.load_state_dict(torch.load(PATH))
        print("model loaded.")
        
    def chat(self):
        while True:
            usr_sent = input("user > ")

            if usr_sent == "끝": break
            if usr_sent == "초기화":
                history = []
                continue
            if len(self.history) == self.max_turns * 2:
                self.history = self.history[2: ]
                self.history[0] = self.bos_token + self.history[0]
            
            usr_sent_pull = self.usr_token + usr_sent + self.sys_token
            if len(self.history) == 0:
                usr_sent_pull = self.bos_token + usr_sent_pull

            self.history.append(usr_sent_pull)

            input_id = self.tokenizer.encode(''.join(self.history), return_tensors="pt").to(self.device)

            with torch.no_grad():
                output_id = model.generate(
                            input_id,
                            max_length=1000,
                            do_samples=True,
                            temperature=0.8,
                            top_k=100,
                            top_p=0.95,
                            max_new_tokens=30,
                            eos_token_id=self.tokenizer.eos_token_id
                )
                
            output = self.tokenizer.decode(output_id[0])
            sys_sent = output.split('<sys>')[-1][: -4]
            print(f'Chatbot > {sys_sent}')
            
            self.history.append(sys_sent)
            print(len(self.history)//2)


In [30]:
chathuman = Train(model, tokenizer, Config)

In [31]:
chathuman.train(Config.epochs, train_dataloader, valid_dataloader)


 Epoch 1/2


KeyboardInterrupt: 

In [51]:
PATH = 'models/model.pt'
chathuman.save(PATH)

model saved.


In [ ]:
plt.plot(chathuman.losses)
plt.plot(chathuman.val_losses)
plt.show()